In [123]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Embedding,Dense ,Lambda
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential

 a. Data preparation

In [124]:
with open("./text.txt","r") as file:
    text=file.read()

Preprocessing text

In [125]:
sentences=text.split('.')
sentences=[s.lower() for s in sentences]
sentences

['deep learning is a subset of machine learning that focuses on utilizing neural networks to perform tasks such as classification, regression, and representation learning',
 ' the field takes inspiration from biological neuroscience and is centered around stacking artificial neurons into layers and "training" them to process data',
 ' the adjective "deep" refers to the use of multiple layers (ranging from three to several hundred or thousands) in the network',
 ' methods used can be either supervised, semi-supervised or unsupervised',
 ' some common deep learning network architectures include fully connected networks, deep belief networks, recurrent neural networks, convolutional neural networks, generative adversarial networks, transformers, and neural radiance fields',
 ' these architectures have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, 

In [126]:
# Tokenizing the sentences
tokenizer=Tokenizer()
tokenizer.fit_on_texts(sentences)
sequences=tokenizer.texts_to_sequences(sentences)
print(sequences)

[[6, 7, 10, 27, 28, 8, 11, 7, 12, 29, 30, 31, 4, 1, 3, 32, 33, 34, 13, 35, 36, 2, 37, 7], [5, 38, 39, 40, 14, 15, 41, 2, 10, 42, 43, 44, 45, 46, 47, 16, 2, 48, 49, 3, 50, 51], [5, 52, 6, 53, 3, 5, 54, 8, 55, 16, 56, 14, 57, 3, 58, 59, 17, 60, 9, 5, 18], [61, 62, 63, 64, 65, 19, 66, 19, 17, 67], [20, 68, 6, 7, 18, 21, 69, 70, 71, 1, 6, 72, 1, 73, 4, 1, 74, 4, 1, 75, 76, 1, 77, 2, 4, 78, 22], [79, 21, 23, 80, 81, 3, 22, 82, 83, 84, 85, 86, 87, 88, 24, 11, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 2, 100, 101, 102, 103, 104, 23, 105, 106, 107, 3, 2, 9, 20, 108, 109, 25, 110, 111], [112, 113, 8, 4, 1, 114, 115, 116, 117, 24, 2, 118, 119, 120, 9, 15, 121, 122, 5, 25, 26], [123, 124, 4, 1, 125, 126, 127, 3, 128, 5, 26, 129, 8, 130, 2, 131, 132, 133, 13, 134, 135, 136, 137, 12, 138], []]


In [127]:
index_to_word=tokenizer.index_word
word_to_index=tokenizer.word_index
print(index_to_word,"\n")
print(word_to_index)

{1: 'networks', 2: 'and', 3: 'to', 4: 'neural', 5: 'the', 6: 'deep', 7: 'learning', 8: 'of', 9: 'in', 10: 'is', 11: 'machine', 12: 'that', 13: 'as', 14: 'from', 15: 'biological', 16: 'layers', 17: 'or', 18: 'network', 19: 'supervised', 20: 'some', 21: 'architectures', 22: 'fields', 23: 'have', 24: 'processing', 25: 'human', 26: 'brain', 27: 'a', 28: 'subset', 29: 'focuses', 30: 'on', 31: 'utilizing', 32: 'perform', 33: 'tasks', 34: 'such', 35: 'classification', 36: 'regression', 37: 'representation', 38: 'field', 39: 'takes', 40: 'inspiration', 41: 'neuroscience', 42: 'centered', 43: 'around', 44: 'stacking', 45: 'artificial', 46: 'neurons', 47: 'into', 48: 'training', 49: 'them', 50: 'process', 51: 'data', 52: 'adjective', 53: 'refers', 54: 'use', 55: 'multiple', 56: 'ranging', 57: 'three', 58: 'several', 59: 'hundred', 60: 'thousands', 61: 'methods', 62: 'used', 63: 'can', 64: 'be', 65: 'either', 66: 'semi', 67: 'unsupervised', 68: 'common', 69: 'include', 70: 'fully', 71: 'connected

b. Generate training data ||
Creating contexts and targets

In [128]:
vocab_size=len(word_to_index)+1
emb_size=100
context_size=2
contexts=[]
targets=[]
for sequence in sequences:
  for i in range(context_size,len(sequence)-context_size):
    context=[sequence[i-2],sequence[i-1],sequence[i+1],sequence[i+2]]
    target=sequence[i]
    contexts.append(context)
    targets.append(target)

print(contexts,"\n")
print(targets)


[[6, 7, 27, 28], [7, 10, 28, 8], [10, 27, 8, 11], [27, 28, 11, 7], [28, 8, 7, 12], [8, 11, 12, 29], [11, 7, 29, 30], [7, 12, 30, 31], [12, 29, 31, 4], [29, 30, 4, 1], [30, 31, 1, 3], [31, 4, 3, 32], [4, 1, 32, 33], [1, 3, 33, 34], [3, 32, 34, 13], [32, 33, 13, 35], [33, 34, 35, 36], [34, 13, 36, 2], [13, 35, 2, 37], [35, 36, 37, 7], [5, 38, 40, 14], [38, 39, 14, 15], [39, 40, 15, 41], [40, 14, 41, 2], [14, 15, 2, 10], [15, 41, 10, 42], [41, 2, 42, 43], [2, 10, 43, 44], [10, 42, 44, 45], [42, 43, 45, 46], [43, 44, 46, 47], [44, 45, 47, 16], [45, 46, 16, 2], [46, 47, 2, 48], [47, 16, 48, 49], [16, 2, 49, 3], [2, 48, 3, 50], [48, 49, 50, 51], [5, 52, 53, 3], [52, 6, 3, 5], [6, 53, 5, 54], [53, 3, 54, 8], [3, 5, 8, 55], [5, 54, 55, 16], [54, 8, 16, 56], [8, 55, 56, 14], [55, 16, 14, 57], [16, 56, 57, 3], [56, 14, 3, 58], [14, 57, 58, 59], [57, 3, 59, 17], [3, 58, 17, 60], [58, 59, 60, 9], [59, 17, 9, 5], [17, 60, 5, 18], [61, 62, 64, 65], [62, 63, 65, 19], [63, 64, 19, 66], [64, 65, 66, 19

In [129]:
# Printing features with target
for i in range(5):
  words=[]
  target=index_to_word.get(targets[i])
  for j in contexts[i]:
    words.append(index_to_word.get(j))
  print(words,"=>",target)

['deep', 'learning', 'a', 'subset'] => is
['learning', 'is', 'subset', 'of'] => a
['is', 'a', 'of', 'machine'] => subset
['a', 'subset', 'machine', 'learning'] => of
['subset', 'of', 'learning', 'that'] => machine


Creating training and testing data


In [ ]:
x=np.array(contexts)
y=np.array(targets)
print("Shape of x:", x.shape)
print("Shape of y:", y.shape)

Shape of x: (164, 4)
Shape of y: (164,)


c. Train model  || *Defining model*

In [132]:
model=Sequential([
    Embedding(input_dim=vocab_size,output_dim=emb_size,input_length=context_size*2),
    Lambda(lambda x:tf.reduce_mean(x,axis=1)),
    Dense(256,activation='relu'),
    Dense(512,activation='relu'),
    Dense(vocab_size,activation='softmax')
])

c:\Users\Atharva\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [133]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [134]:
history=model.fit(x,y,epochs=20)

Epoch 1/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0117 - loss: 4.9356   
Epoch 2/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1009 - loss: 4.9130 
Epoch 3/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1098 - loss: 4.8821 
Epoch 4/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0918 - loss: 4.8244 
Epoch 5/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0610 - loss: 4.7458 
Epoch 6/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0807 - loss: 4.5578 
Epoch 7/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0699 - loss: 4.4364 
Epoch 8/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0858 - loss: 4.2873 
Epoch 9/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1204 - loss: 4.1339 
Epoch 10/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1174 - loss: 4.0791 
Epoch 11/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1436 - loss: 3.8973 
Epoch 12/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1462 - loss: 3.6919 

 d. Output

In [135]:
test_words=[index_to_word[index] for index in contexts[0]]

input_data=np.expand_dims(contexts[0],axis=0)
print(input_data)

pred=model.predict(input_data)
predicted_index=np.argmax(pred[0])

print("Context words:", test_words)
print("Predicted terget word : ",index_to_word[predicted_index])


[[ 6  7 27 28]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Context words: ['deep', 'learning', 'a', 'subset']
Predicted terget word :  is
